In [1]:
from nltk.corpus import treebank

import numpy as np

from sklearn.model_selection import cross_val_score, GroupKFold
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

In [2]:
# training data
# note the differences between this and our text. Review text can be messier. It's not always proper english or proofread
tagged_sentences = treebank.tagged_sents()
tagged_sentences[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [3]:
def fw_extract_features(tagged_sentence, index):
    token, tag = tagged_sentence[index]
    prev_token = ""
    prev_tag = ""
    if index > 0:
        prev_token, prev_tag = tagged_sentence[index - 1]
            
    is_number = False
    try:
        if float(token):
            is_number = True
    except:
        pass
    features_dict = {"token": token
        , "lower_cased_token": token.lower()
        , "prev_token": prev_token
        , "prev_tag": prev_tag
        , "suffix1": token[-1]
        , "suffix2": token[-2:]
        , "suffix3": token[-3:]
        , "is_capitalized": token.upper() == token
        , "is_number": is_number}
    return features_dict

In [4]:
fw_extract_features(tagged_sentences[0], 3)

{'token': '61',
 'lower_cased_token': '61',
 'prev_token': ',',
 'prev_tag': ',',
 'suffix1': '1',
 'suffix2': '61',
 'suffix3': '61',
 'is_capitalized': True,
 'is_number': True}

In [5]:
from sklearn.feature_extraction import DictVectorizer
X_features = []
for sentence in tagged_sentences:
    for k in range(len(sentence)):
        X_features.append(fw_extract_features(sentence, k))

vectoriser = DictVectorizer(sparse=False)
X = vectoriser.fit_transform(X_features)

In [6]:
Y = []
for sentence in tagged_sentences:
    for k in range(len(sentence)):
        Y.append(sentence[k][1])

In [7]:
from sklearn import tree
import time

t0=time.time()
clf = tree.DecisionTreeClassifier(criterion='entropy')
clf.fit(X, Y)
t1=time.time()
print(t1-t0)

1603.178388595581


In [8]:
import pickle
with open('pos_tagger.clf',mode='wb') as f:
    pickle.dump(clf,f)

with open('pos_tagger.vec',mode='wb') as f:
    pickle.dump(vectoriser,f)


In [12]:
import time
from sklearn import tree

t0 = time.time()
clf = tree.DecisionTreeClassifier(criterion='entropy')
scores = cross_val_score(clf, X, Y, cv=5, n_jobs=1)
t1 = time.time()
print(t1-t0)

/projects/bope9760/.conda/envs/mlclass/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


2526.7728765010834


In [15]:
scores

array([0.91189909, 0.90812019, 0.90856717, 0.91666253, 0.91894711])